### Low Depth Network
AhmadReza Nopoush

 id:  610301194

 

In [34]:
import numpy as np
from sklearn.datasets import make_gaussian_quantiles
from sklearn.model_selection import train_test_split

at the begining we produce the data we need.

In [35]:
np.random.seed(42)
Xs , y = make_gaussian_quantiles(n_samples=300,n_classes=2, n_features=8, random_state=42)
X_train , X_test , y_train , y_test = train_test_split(Xs,y, test_size=0.2, random_state=42)
y_train = np.reshape(y_train,newshape=(240,1))
y_test = np.reshape(y_test,newshape=(60,1))

Now, we define the activation function and their deriativision wich used in network.

In [36]:
def Sigmoid(Z) -> float:
    return 1 / (1 + np.exp(-Z))

def RelU(Z) -> float:
    return np.maximum(0, Z)
    
def deriative_of_RelU(Z) -> int:
    return Z > 0
    
def deriative_of_Sigmoid(Z) -> float:
    s = Sigmoid(Z)
    return s*(1-s)

according to the formulas we have, we define a "ForwardPropagation" function.

$$
Z_1 = W_1 \times X^T 
$$
$$ 
A_1 = \text{ReLU}(Z_1) 
$$
$$ 
Z_2 = W_2 \times A_1 
$$
$$ 
A_2 = \sigma(Z_2) = Y_{\text{pred}} 
$$


In [37]:
def Predict(X, We1, b1, We2, b2,threshold=0.5):
    Zi1 = np.dot(We1, X.T) + b1
    Ay1 = RelU(Zi1)
    Zi2 = np.dot(We2, Ay1) + b2
    Ay2 = Sigmoid(Zi2)
    return (Ay2 >= threshold).astype(int)

Now, we should implement backward propagation using mathematic formulas we have below.

$$
\frac{\partial \text{cost}}{\partial W_2} = \frac{\partial \text{cost}}{\partial A_2} \times \frac{\partial A_2}{\partial Z_2} \times \frac{\partial Z_2}{\partial W_2} = \left(-\frac{2}{n}(Y_{\text{true}} - A_2) \odot A_2 \odot (1 - A_2)\right) \times A_1^T 
$$

$$
\frac{\partial \text{cost}}{\partial W_1} = \frac{\partial \text{cost}}{\partial A_2} \times \frac{\partial A_2}{\partial Z_2} \times \frac{\partial Z_2}{\partial A_1} \times \frac{\partial A_1}{\partial Z_1} \times \frac{\partial Z_1}{\partial W_1} = \left(\left(\left(-\frac{2}{n}(Y_{\text{true}} - A_2) \odot A_2 \odot (1 - A_2)\right)^T \times W_2\right) \odot \frac{\partial A_1}{\partial Z_1}\right) \times X 
$$


In [38]:

def BackPropagation(Xs, Yp, Zi1, Ay1, Zi2, Ay2, We1, We2):
    n = len(Xs)
    
    dZ2 = (2/n)*(Ay2 - Yp.T)*deriative_of_Sigmoid(Ay2)
    dW2 = np.dot(dZ2, Ay1.T)
    db2 = np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(We2.T, dZ2)
    dZ1 = dA1 * deriative_of_RelU(Zi1)
    dW1 = np.dot(dZ1, Xs)
    db1 = np.sum(dZ1, axis=1, keepdims=True)
    
    return dW1, db1, dW2, db2

Now, the updating rules is obvieous. we defined the **Update** function according to formulas.

In [39]:
def Update(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

And of course, the implementation of MSE; our loss function.

In [40]:
def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

Now, we define the **Train** function, to train our model based on our data. This function get the paramethers:
1. **X_train**: the list of 6D-arrays wich contains the train dataset.
2. **Y_train**: the lable of train dataset.
3. **hidden**: the number of neurons in hidden layer.
4. **learning_rate**
5. **epochs**: number of iterations to train the model.

In [47]:
def Train(X_train, Y_train, hidden:int, learning_rate:float, epochs:int):
    #count of input neurons:
    inp = len(X_train[0])
    
    #count of output neurons:
    output = 1

    #initializing paramethers:
    Weight1 = np.random.randn(hidden, inp) * 0.01
    w=Weight1
    b1 = np.zeros((hidden, 1))
    Weight2 = np.random.randn(output, hidden) * 0.01
    b2 = np.zeros((output, 1))
    
    #for each epoch do:
    for epx in range(epochs):
        #generate solution and weights
        Zi1 = np.dot(Weight1, X_train.T) + b1
        Aa1 = RelU(Zi1)
        Zi2 = np.dot(Weight2, Aa1) + b2
        Ay2 = Sigmoid(Zi2)
        Ay2 = (Ay2 >= 0.5).astype(int)
                        
        #calculate the MSE
        loss = MSE(Y_train, Ay2.T)
        
        #back propagation
        dW1, db1, dW2, db2 = BackPropagation(X_train, Y_train, Zi1, Aa1, Zi2, Ay2, Weight1, Weight2)
        
        #update weights
        We1, bi1, We2, bi2 = Update(Weight1, b1, Weight2, b2, dW1, db1, dW2, db2, learning_rate)
        Weight1 = We1
        Weight2 = We2
        b1,b2 = bi1,bi2
        
        #print loss every 10 epochs
        if epx % 50 == 0:
            print("Epoch ",epx,": Training loss = ", loss)
    
    #return weights
    return Weight1, b1, Weight2, b2

at last, we test the model we implemented.

In [54]:
epochs = 1000
learning_rate = 0.01

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.5
Epoch  50 : Training loss =  0.44166666666666665
Epoch  100 : Training loss =  0.35
Epoch  150 : Training loss =  0.25833333333333336
Epoch  200 : Training loss =  0.20833333333333334
Epoch  250 : Training loss =  0.12916666666666668
Epoch  300 : Training loss =  0.13333333333333333
Epoch  350 : Training loss =  0.1125
Epoch  400 : Training loss =  0.08333333333333333
Epoch  450 : Training loss =  0.04583333333333333
Epoch  500 : Training loss =  0.0375
Epoch  550 : Training loss =  0.04583333333333333
Epoch  600 : Training loss =  0.008333333333333333
Epoch  650 : Training loss =  0.0125
Epoch  700 : Training loss =  0.0
Epoch  750 : Training loss =  0.0
Epoch  800 : Training loss =  0.0
Epoch  850 : Training loss =  0.0
Epoch  900 : Training loss =  0.0
Epoch  950 : Training loss =  0.0
Test Loss: 0.05
Test Accuracy: 0.95


In [58]:
epochs = 500
learning_rate = 0.01

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.475
Epoch  50 : Training loss =  0.3416666666666667
Epoch  100 : Training loss =  0.24583333333333332
Epoch  150 : Training loss =  0.20416666666666666
Epoch  200 : Training loss =  0.16666666666666666
Epoch  250 : Training loss =  0.125
Epoch  300 : Training loss =  0.09166666666666666
Epoch  350 : Training loss =  0.0625
Epoch  400 : Training loss =  0.08333333333333333
Epoch  450 : Training loss =  0.06666666666666667
Test Loss: 0.13333333333333333
Test Accuracy: 0.8666666666666667


In [59]:
epochs = 100
learning_rate = 0.01

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.49166666666666664
Epoch  50 : Training loss =  0.3541666666666667
Test Loss: 0.26666666666666666
Test Accuracy: 0.7333333333333333


#### Conclusion ####
By increasing the number of Epochs, the model fits better with the data. When the Epochs reach 1000, the network completely matches the dataset and the MSE becomes zero. So the number of Epochs more than 1000 will not be meaningful. To check the effect of the learning coefficient, we consider the **Epochs = 500**,

In [61]:
epochs = 500
learning_rate = 0.001

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.48333333333333334
Epoch  50 : Training loss =  0.5458333333333333
Epoch  100 : Training loss =  0.5333333333333333
Epoch  150 : Training loss =  0.5208333333333334
Epoch  200 : Training loss =  0.525
Epoch  250 : Training loss =  0.5291666666666667
Epoch  300 : Training loss =  0.5083333333333333
Epoch  350 : Training loss =  0.5041666666666667
Epoch  400 : Training loss =  0.49583333333333335
Epoch  450 : Training loss =  0.49583333333333335
Test Loss: 0.6333333333333333
Test Accuracy: 0.36666666666666664


In [62]:
epochs = 500
learning_rate = 0.005

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.4791666666666667
Epoch  50 : Training loss =  0.4708333333333333
Epoch  100 : Training loss =  0.4125
Epoch  150 : Training loss =  0.32083333333333336
Epoch  200 : Training loss =  0.2833333333333333
Epoch  250 : Training loss =  0.2791666666666667
Epoch  300 : Training loss =  0.26666666666666666
Epoch  350 : Training loss =  0.23333333333333334
Epoch  400 : Training loss =  0.2
Epoch  450 : Training loss =  0.15416666666666667
Test Loss: 0.3
Test Accuracy: 0.7


In [63]:
epochs = 500
learning_rate = 0.01

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.44166666666666665
Epoch  50 : Training loss =  0.35
Epoch  100 : Training loss =  0.2375
Epoch  150 : Training loss =  0.1625
Epoch  200 : Training loss =  0.1125
Epoch  250 : Training loss =  0.09583333333333334
Epoch  300 : Training loss =  0.11666666666666667
Epoch  350 : Training loss =  0.07916666666666666
Epoch  400 : Training loss =  0.075
Epoch  450 : Training loss =  0.07083333333333333
Test Loss: 0.06666666666666667
Test Accuracy: 0.9333333333333333


In [64]:
epochs = 500
learning_rate = 0.05

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.5083333333333333
Epoch  50 : Training loss =  0.2875
Epoch  100 : Training loss =  0.09166666666666666
Epoch  150 : Training loss =  0.05416666666666667
Epoch  200 : Training loss =  0.016666666666666666
Epoch  250 : Training loss =  0.008333333333333333
Epoch  300 : Training loss =  0.0
Epoch  350 : Training loss =  0.0
Epoch  400 : Training loss =  0.0
Epoch  450 : Training loss =  0.0
Test Loss: 0.05
Test Accuracy: 0.95


In [65]:
epochs = 500
learning_rate = 0.1

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.5166666666666667
Epoch  50 : Training loss =  0.30416666666666664
Epoch  100 : Training loss =  0.25833333333333336
Epoch  150 : Training loss =  0.0125
Epoch  200 : Training loss =  0.0
Epoch  250 : Training loss =  0.0
Epoch  300 : Training loss =  0.0
Epoch  350 : Training loss =  0.0
Epoch  400 : Training loss =  0.0
Epoch  450 : Training loss =  0.0
Test Loss: 0.03333333333333333
Test Accuracy: 0.9666666666666667


In [66]:
epochs = 500
learning_rate = 0.5

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.4625
Epoch  50 : Training loss =  0.475
Epoch  100 : Training loss =  0.029166666666666667
Epoch  150 : Training loss =  0.0
Epoch  200 : Training loss =  0.0
Epoch  250 : Training loss =  0.0
Epoch  300 : Training loss =  0.0
Epoch  350 : Training loss =  0.0
Epoch  400 : Training loss =  0.0
Epoch  450 : Training loss =  0.0
Test Loss: 0.03333333333333333
Test Accuracy: 0.9666666666666667


In [67]:
epochs = 500
learning_rate = 1

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.48333333333333334
Epoch  50 : Training loss =  0.3541666666666667
Epoch  100 : Training loss =  0.0
Epoch  150 : Training loss =  0.0
Epoch  200 : Training loss =  0.0
Epoch  250 : Training loss =  0.0
Epoch  300 : Training loss =  0.0
Epoch  350 : Training loss =  0.0
Epoch  400 : Training loss =  0.0
Epoch  450 : Training loss =  0.0
Test Loss: 0.05
Test Accuracy: 0.95


In [69]:
epochs = 500
learning_rate = 5

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.5166666666666667
Epoch  50 : Training loss =  0.2
Epoch  100 : Training loss =  0.0
Epoch  150 : Training loss =  0.0
Epoch  200 : Training loss =  0.0
Epoch  250 : Training loss =  0.0
Epoch  300 : Training loss =  0.0
Epoch  350 : Training loss =  0.0
Epoch  400 : Training loss =  0.0
Epoch  450 : Training loss =  0.0
Test Loss: 0.13333333333333333
Test Accuracy: 0.8666666666666667


In [70]:
epochs = 500
learning_rate = 10

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.5166666666666667
Epoch  50 : Training loss =  0.3125
Epoch  100 : Training loss =  0.0
Epoch  150 : Training loss =  0.0
Epoch  200 : Training loss =  0.0
Epoch  250 : Training loss =  0.0
Epoch  300 : Training loss =  0.0
Epoch  350 : Training loss =  0.0
Epoch  400 : Training loss =  0.0
Epoch  450 : Training loss =  0.0
Test Loss: 0.23333333333333334
Test Accuracy: 0.7666666666666667


#### Concolusion ####
As the learning_rate increases, the speed of weight change increases and MSE approaches zero at a higher rate. Seeing the number of the Epochs also confirms our claim. But by increasing the size too much, we will see that the fitness on the test data decreases, and this indicates the overfitting of the model.

In [71]:
epochs = 1000
learning_rate = 1

weights1, b1, weights2, b2 = Train(X_train, y_train, 1000, learning_rate, epochs)

Y_pred = Predict(X_test, weights1, b1, weights2, b2)
test_loss = MSE(y_test, Y_pred.T)
print(f"Test Loss: {test_loss}")

accuracy = np.mean(Y_pred.T == y_test)
print(f"Test Accuracy: {accuracy}")

Epoch  0 : Training loss =  0.4791666666666667
Epoch  50 : Training loss =  0.44583333333333336
Epoch  100 : Training loss =  0.0
Epoch  150 : Training loss =  0.0
Epoch  200 : Training loss =  0.0
Epoch  250 : Training loss =  0.0
Epoch  300 : Training loss =  0.0
Epoch  350 : Training loss =  0.0
Epoch  400 : Training loss =  0.0
Epoch  450 : Training loss =  0.0
Epoch  500 : Training loss =  0.0
Epoch  550 : Training loss =  0.0
Epoch  600 : Training loss =  0.0
Epoch  650 : Training loss =  0.0
Epoch  700 : Training loss =  0.0
Epoch  750 : Training loss =  0.0
Epoch  800 : Training loss =  0.0
Epoch  850 : Training loss =  0.0
Epoch  900 : Training loss =  0.0
Epoch  950 : Training loss =  0.0
Test Loss: 0.05
Test Accuracy: 0.95
